In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
import mlflow

In [2]:
mlflow.set_tracking_uri("https://dagshub.com/Naman12769/uber-demand-prediction.mlflow")

In [3]:
import dagshub
dagshub.init(repo_owner='Naman12769', repo_name='uber-demand-prediction', mlflow=True)


Accessing as Naman12769

Initialized MLflow to track repo "Naman12769/uber-demand-prediction"

Repository Naman12769/uber-demand-prediction initialized!

In [4]:

# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

In [5]:
train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [6]:
train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [7]:
test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [8]:
X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]


In [9]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [10]:
#  make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [11]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [12]:

from sklearn import set_config

set_config(transform_output="pandas")

In [13]:

# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [14]:
encoder

ColumnTransformer(force_int_remainder_cols=False, n_jobs=-1,
                  remainder='passthrough',
                  transformers=[('ohe',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['region', 'day_of_week'])])

In [15]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

In [16]:
import optuna
import tqdm

c:\Users\hi\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
mlflow.set_experiment("Model Selection")

2025/06/10 17:22:11 INFO mlflow.tracking.fluent: Experiment with name 'Model Selection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e1eee2f2bbb44cca9551bace09137416', creation_time=1749556332676, experiment_id='0', last_update_time=1749556332676, lifecycle_stage='active', name='Model Selection', tags={}>

In [21]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())
        
        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [22]:
with mlflow.start_run(run_name="best_model",nested=True) as parent:
  study=optuna.create_study(study_name="model_selection",direction="minimize")
  study.optimize(func=objective,n_trials=50,n_jobs=-1)
  mlflow.log_params(study.best_params)
  mlflow.log_metric("BEST MAPE",study.best_value)

[I 2025-06-10 17:38:32,438] A new study created in memory with name: model_selection


🏃 View run funny-hare-572 at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0/runs/abc35bfaf19147649d3b3651679229cb
🧪 View experiment at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0
🏃 View run defiant-panda-563 at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0/runs/04d662f7865d481d9d782869a7fdce46
🧪 View experiment at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0
🏃 View run adaptable-loon-970 at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0/runs/982727e060444fadbbe9bdabadf7b45b
🧪 View experiment at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-06-10 17:38:41,421] Trial 2 finished with value: 0.0793479028546311 and parameters: {'model_name': 'LR'}. Best is trial 2 with value: 0.0793479028546311.
[I 2025-06-10 17:38:43,531] Trial 1 finished with value: 0.0793479028546311 and parameters: {'model_name': 'LR'}. Best is trial 2 with value: 0.0793479028546311.
[I 2025-06-10 17:38:44,439] Trial 0 finished with value: 0.0793479028546311 and parameters: {'model_name': 'LR'}. Best is trial 2 with value: 0.0793479028546311.


🏃 View run capricious-foal-299 at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0/runs/168f73605c8847cea55247d2811d3fee
🧪 View experiment at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-06-10 17:38:50,658] Trial 3 finished with value: 0.415421724319458 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 40, 'learning_rate_xgb': 0.08841389736787204, 'max_depth_xgb': 3}. Best is trial 2 with value: 0.0793479028546311.


...

🏃 View run brawny-stork-79 at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0/runs/f8f647bc6f7b4014b7c5e8496f29475f
🧪 View experiment at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-06-10 17:44:45,912] Trial 47 finished with value: 0.0793479028546311 and parameters: {'model_name': 'LR'}. Best is trial 2 with value: 0.0793479028546311.
[I 2025-06-10 17:44:46,886] Trial 48 finished with value: 0.0793479028546311 and parameters: {'model_name': 'LR'}. Best is trial 2 with value: 0.0793479028546311.


🏃 View run blushing-kite-974 at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0/runs/e033669c39db4c0b907f5e53f4692691
🧪 View experiment at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0


[I 2025-06-10 17:44:49,899] Trial 49 finished with value: 0.0793479028546311 and parameters: {'model_name': 'LR'}. Best is trial 2 with value: 0.0793479028546311.


🏃 View run best_model at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0/runs/f9cbb597bd01409187ed52dafb3fcd33
🧪 View experiment at: https://dagshub.com/Naman12769/uber-demand-prediction.mlflow/#/experiments/0


In [23]:

# best value

study.best_value

0.0793479028546311

In [24]:

# best parameters

study.best_params

{'model_name': 'LR'}

In [25]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

params_model_name
LR      28
GBR     10
RF       7
XGBR     5
Name: count, dtype: int64

In [26]:
from optuna.visualization import (
    plot_optimization_history, 
    plot_parallel_coordinate, 
    plot_param_importances
)

In [27]:
plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'y': [0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.415421724319458, 6.361350745022831, 0.8282738296617987,
                    0.0793479028546311, 6.24914347976934, 6.361810684204102,
                    0.17650979550655893, 6.536756028818556, 0.24126300873877268,
                    0.3467551195659892, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    6.512824058532715, 0.13029011350572198, 6.508543968200684,
                    0.0793479028546311, 6.490295886993408, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.5443209920941997, 0.5443209920941997,
                    0.5443209920941997, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.5443209920941997, 0.0793479028546311,
                    6.291502827557266, 6.091215486436146, 6.099819225648569,
                    6.322555425323046, 6.05066441767426, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311]},
             {'mode': 'lines',
              'name': 'Best Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              'y': [0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311, 0.0793479028546311,
                    0.0793479028546311, 0.0793479028546311]},
             {'marker': {'color': '#cccccc'},
              'mode': 'markers',
              'name': 'Infeasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Optimization History Plot'},
               'xaxis': {'title': {'text': 'Trial'}},
               'yaxis': {'title': {'text': 'Objective Value'}}}
})

In [28]:

plot_parallel_coordinate(study, params=["model_name"])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'dimensions': [{'label': 'Objective Value',
                              'range': [0.0793479028546311, 6.536756028818556],
                              'values': [0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.415421724319458,
                                         6.361350745022831, 0.8282738296617987,
                                         0.0793479028546311, 6.24914347976934,
                                         6.361810684204102, 0.17650979550655893,
                                         6.536756028818556, 0.24126300873877268,
                                         0.3467551195659892, 0.0793479028546311,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 6.512824058532715,
                                         0.13029011350572198, 6.508543968200684,
                                         0.0793479028546311, 6.490295886993408,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.5443209920941997,
                                         0.5443209920941997, 0.5443209920941997,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.5443209920941997,
                                         0.0793479028546311, 6.291502827557266,
                                         6.091215486436146, 6.099819225648569,
                                         6.322555425323046, 6.05066441767426,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.0793479028546311,
                                         0.0793479028546311, 0.0793479028546311]},
                             {'label': 'model_name',
                              'range': [0, 3],
                              'ticktext': [LR, XGBR, GBR, RF],
                              'tickvals': [0, 1, 2, 3],
                              'values': [0, 0, 0, 1, 2, 2, 0, 2, 1, 3, 2, 2, 3, 0,
                                         0, 0, 0, 0, 0, 0, 0, 1, 3, 1, 0, 1, 0, 0,
                                         0, 0, 0, 3, 3, 3, 0, 0, 0, 3, 0, 2, 2, 2,
                                         2, 2, 0, 0, 0, 0, 0, 0]}],
              'labelangle': 30,
              'labelside': 'bottom',
              'line': {'color': [0.0793479028546311, 0.0793479028546311,
                                 0.0793479028546311, 0.415421724319458,
                                 6.361350745022831, 0.8282738296617987,
                                 0.0793479028546311, 6.24914347976934,
                                 6.361810684204102, 0.17650979550655893,
                                 6.536756028818556, 0.24126300873877268,
                                 0.3467551195659892, 0.0793479028546311,
                                 0.0793479028546311, 0.0793479028546311,
                                 0.0793479028546311, 0.0793479028546311,
                                 0.0793479028546311, 0.0793479028546311,
                                 0.0793479028546311, 6.512824058532715,
                                 0.13029011350572198, 6.508543968200684,
                                 0.0793479028546311, 6.490295886993408,
                                 0.0793479028546311, 0.0793479028546311,
                                 0.0793479028546311, 0.0793479028546311,
                                 0.0793479028546311, 0.5443209920941997,
                                 0.5443209920941997, 0.5443209920941997

In [29]:

# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded) 
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

The training error is  0.08778013304566602
The test error is  0.0793479028546311


In [30]:
lr.coef_

array([-2.33737604,  0.71512405, -0.55601505, -1.25311068, -3.20463231,
       -0.86685973, -2.79925402, -3.62516859,  0.41386463, -2.9376376 ,
       -1.97624678, -3.75050442,  0.51806283, -2.54033388, -2.43297463,
        0.47632075,  0.61254786, -4.7417372 , -2.03077217, -1.26960984,
       -4.03690273, -2.08863167, -1.0414428 ,  0.73561736, -0.99999442,
       -0.85944985, -2.43098478,  0.67112238,  0.57385071, -0.11719951,
       -0.28045898, -0.37180749, -0.5238324 , -0.4233113 , -0.34045774,
       -0.54170892, -0.36264553, -0.2493965 , -0.31905518,  2.4912456 ])

In [31]:

def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss

In [32]:
study = optuna.create_study(study_name="tune_model", direction="minimize")

[I 2025-06-10 17:48:13,209] A new study created in memory with name: tune_model


In [33]:

# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

[I 2025-06-10 17:48:17,458] Trial 1 finished with value: 0.07921972105922186 and parameters: {'alpha': 60.674787242813174}. Best is trial 1 with value: 0.07921972105922186.
[I 2025-06-10 17:48:17,551] Trial 0 finished with value: 0.07925458705716804 and parameters: {'alpha': 40.68586307441356}. Best is trial 1 with value: 0.07921972105922186.
[I 2025-06-10 17:48:17,624] Trial 2 finished with value: 0.07919568917240144 and parameters: {'alpha': 76.50578547095813}. Best is trial 2 with value: 0.07919568917240144.


Best trial: 4. Best value: 0.0791733:   5%|▌         | 5/100 [00:05<01:52,  1.19s/it]

[I 2025-06-10 17:48:19,469] Trial 3 finished with value: 0.07922769502339565 and parameters: {'alpha': 55.79002528642387}. Best is trial 2 with value: 0.07919568917240144.
[I 2025-06-10 17:48:19,492] Trial 4 finished with value: 0.07917328839154779 and parameters: {'alpha': 92.70202738858858}. Best is trial 4 with value: 0.07917328839154779.
[I 2025-06-10 17:48:19,500] Trial 6 finished with value: 0.07921375400582292 and parameters: {'alpha': 64.4487047252864}. Best is trial 4 with value: 0.07917328839154779.


Best trial: 4. Best value: 0.0791733:   8%|▊         | 8/100 [00:05<00:59,  1.54it/s]

[I 2025-06-10 17:48:20,341] Trial 5 finished with value: 0.0792179410706045 and parameters: {'alpha': 61.78950662774614}. Best is trial 4 with value: 0.07917328839154779.
[I 2025-06-10 17:48:20,355] Trial 7 finished with value: 0.07920823563405295 and parameters: {'alpha': 68.0449594423406}. Best is trial 4 with value: 0.07917328839154779.
[I 2025-06-10 17:48:20,359] Trial 8 finished with value: 0.07926888643550939 and parameters: {'alpha': 33.36003477976577}. Best is trial 4 with value: 0.07917328839154779.


...

[I 2025-06-10 17:48:42,423] Trial 89 finished with value: 0.0791755668357908 and parameters: {'alpha': 90.97540522939651}. Best is trial 67 with value: 0.07916407581574215.
[I 2025-06-10 17:48:42,444] Trial 91 finished with value: 0.07917552518963059 and parameters: {'alpha': 91.00684109552694}. Best is trial 67 with value: 0.07916407581574215.
[I 2025-06-10 17:48:42,484] Trial 93 finished with value: 0.07917202599677427 and parameters: {'alpha': 93.66806585659315}. Best is trial 67 with value: 0.07916407581574215.
[I 2025-06-10 17:48:42,489] Trial 92 finished with value: 0.07917259399446783 and parameters: {'alpha': 93.23294429166918}. Best is trial 67 with value: 0.07916407581574215.


Best trial: 67. Best value: 0.0791641:  98%|█████████▊| 98/100 [00:28<00:00,  7.32it/s]

[I 2025-06-10 17:48:43,069] Trial 94 finished with value: 0.07916440712407302 and parameters: {'alpha': 99.72177602427786}. Best is trial 67 with value: 0.07916407581574215.
[I 2025-06-10 17:48:43,125] Trial 95 finished with value: 0.07916658494840123 and parameters: {'alpha': 97.95903072486642}. Best is trial 67 with value: 0.07916407581574215.
[I 2025-06-10 17:48:43,129] Trial 96 finished with value: 0.07916634479659379 and parameters: {'alpha': 98.15253092593538}. Best is trial 67 with value: 0.07916407581574215.
[I 2025-06-10 17:48:43,209] Trial 97 finished with value: 0.07916415677396098 and parameters: {'alpha': 99.92569519175657}. Best is trial 67 with value: 0.07916407581574215.


Best trial: 67. Best value: 0.0791641: 100%|██████████| 100/100 [00:29<00:00,  3.43it/s]

[I 2025-06-10 17:48:43,543] Trial 98 finished with value: 0.07926792328606908 and parameters: {'alpha': 33.84070191313728}. Best is trial 67 with value: 0.07916407581574215.
[I 2025-06-10 17:48:43,579] Trial 99 finished with value: 0.07926797199947895 and parameters: {'alpha': 33.81631556028432}. Best is trial 67 with value: 0.07916407581574215.


In [34]:

study.best_params

{'alpha': 99.99164572579834}

In [35]:

study.best_value

0.07916407581574215